In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
df = pd.read_csv('./../../data/processed/ID_Y.csv')
# df = pd.read_csv('./../../data/processed/.csv')

Mbool = 0
sales = df.loc[df.IMPORT == Mbool].groupby(['ID', 'YEAR'])['VART'].sum().unstack()

sales = sales.loc[sales.sum(1).sort_values().index]

logsales = np.log10(sales)
demlogsales = logsales.subtract(logsales.mean(1), axis = 0)

sizes = sales.loc[sales.sum(1).sort_values().index].sum(1)

Q = 10
parts = pd.cut(sizes.cumsum()/sizes.sum(), Q, labels = range(Q)).sort_index()

## Effective Nq
eff_nq = sales.groupby(parts).count().mean(1).round().astype(int)
eff_nq

0    98643
1     4212
2     1414
3      635
4      319
5      168
6       92
7       46
8       17
9        5
dtype: int64

In [3]:
eff_nq[1:].sum()

6908

In [4]:
## Microshocks
demlogsales['parts'] = parts
std_data = demlogsales.loc[demlogsales.iloc[:, :-1].count(1) > 1]

std_info = std_data.reset_index().set_index(['ID', 'parts']).stack()
sdt_q = std_info.groupby(level = 'parts').std()
avg_std = std_info.std()

# Un array de donde samplear shocks empiricos.
emp_shocks = std_info.values
# len(emp_shocks)

In [5]:
print(avg_std)

sdt_q

0.49146737237448623


parts
0    0.481817
1    0.584927
2    0.604532
3    0.640641
4    0.664912
5    0.620892
6    0.610946
7    0.722794
8    0.519787
9    1.395501
dtype: float64

In [6]:
# emp_nqs = np.round(nq.sort_values()).astype(int)
partition = eff_nq.astype(int)
ss = np.arange(0.1, .8, .1)
M = 200

T = 17

In [7]:
# I don't knwo the possible mus, because on every firm I subtracted the observed mean value.
# Still, one can say they are well below mu = 0.1.

emp_shocks.mean()

1.2571764836772036e-18

In [8]:
mus = [0, 0.01, 0.02, 0.05, 0.1]

In [13]:

### Experiments (gaussian and laplace deviations from mean)

results = []

s0 = emp_shocks.std()

for dist in ['norm', 'lapl', 'emp']:
    for q in range(Q)[1:]:
        part = partition.index[q]
        n = partition.values[q]
#         x0 = np.log10(sizes.loc[parts == part]).values # comment for narrow bin limit
        print(q, n)
        for s in ss:
            for mu in mus:

                for m in range(M):
                    if dist == 'norm':
                        shocks = np.random.normal(mu, s, (n, T))
                    elif dist == 'lapl':
                        shocks = np.random.laplace(mu, s/np.sqrt(2), (n, T))
                    elif dist == 'emp':
                        shocks = (mu + np.random.choice(emp_shocks, n * T)*(s/s0)).reshape(n, T)

                    ratio = np.power(10, shocks).sum(0)/n #M?
                    log_ratio = np.log10(ratio) #M?
    #                 values = np.log10(np.power(10, x0[:, None] + shocks).sum(0)/np.power(10, x0[:, None]).sum(0)) #M? # comment for narrow bin limit

    #             t += [np.power(10, x0 + x1).sum()/(n*10**mu0)]
                    results += [[dist, s, mu, n, m, ratio.mean(), ratio.std(), ratio.var(), log_ratio.mean(), log_ratio.std(), log_ratio.var()]]

result = pd.DataFrame(results, columns = ['dist', 's', 'mu', 'nq', 'repeat', 'mean_ratio', 'std_ratio', 'var_ratio', 'mean_log_ratio', 'std_log_ratio', 'var_log_ratio'])


1 4212


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/miglesia/anaconda2/envs/my_pymc_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-aee6c178136e>", line 25, in <module>
    ratio = np.power(10, shocks).sum(0)/n #M?
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/miglesia/anaconda2/envs/my_pymc_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/miglesia/anaconda2/envs/my_pymc_env/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_l

KeyboardInterrupt: 

In [10]:
result.to_csv('./experiment_3.csv', index = False)

In [11]:
result.head()

,dist,s,mu,nq,repeat,mean_ratio,std_ratio,var_ratio,mean_log_ratio,std_log_ratio,var_log_ratio
0,norm,0.1,0.0,4212,0,1.027992,0.004581,0.000021,0.011985,0.001936,0.000004
1,norm,0.1,0.0,4212,1,1.025738,0.004398,0.000019,0.011032,0.001861,0.000003
2,norm,0.1,0.0,4212,2,1.026318,0.002923,0.000009,0.011280,0.001236,0.000002
3,norm,0.1,0.0,4212,3,1.027716,0.003463,0.000012,0.011871,0.001463,0.000002
4,norm,0.1,0.0,4212,4,1.026964,0.002398,0.000006,0.011554,0.001014,0.000001


In [12]:
result.tail()

,dist,s,mu,nq,repeat,mean_ratio,std_ratio,var_ratio,mean_log_ratio,std_log_ratio,var_log_ratio
188995,emp,0.7,0.1,5,195,17.212552,30.667720,940.509075,0.740966,0.597533,0.357046
188996,emp,0.7,0.1,5,196,3.811210,3.621271,13.113606,0.442794,0.340137,0.115693
188997,emp,0.7,0.1,5,197,28.219017,102.976585,10604.176991,0.465511,0.590461,0.348644
188998,emp,0.7,0.1,5,198,2.725077,3.014994,9.090187,0.271304,0.349368,0.122058
188999,emp,0.7,0.1,5,199,5.138255,6.685347,44.693863,0.523427,0.365735,0.133762
